# A dataset of air quality

We are going to use a dataset of air pollution measurements in Beijing archived by the UCI. To read about the dataset visit the [UCI page](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data).

We are going to:
  1. Preprocess the data
  2. Build our own Lasso-regression
  3. Use sklearn's tools to perform regression on the data
  
Let's get started

In [43]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

We will be trying to predict the pollution (PM2.5) using:
- temperature ('TEMP')
- pressure ('PRES')
- dew-point temperature ('DEWP')
- precipitation ('RAIN')
- wind direction ('wd')
- wind speed ('WSPM')

In [44]:
urllib.request.urlretrieve('https://drive.google.com/uc?id=1m1g4Xn1wMAGV_EU0Nh1HTI1ogA3-tqJk&export=download', './data.csv')

('./data.csv', <http.client.HTTPMessage at 0x16403a710>)

In [45]:
raw_df = pd.read_csv('data.csv',index_col='No')

#put the columns in a useful order
raw_df = raw_df[['PM2.5', 'year', 'month', 'day', 'hour', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station']]

Some of the records are missing. We need to handle that before we can easily use the data with most ML tools.

### Removing missing data

We are going to handle this by dropping those rows which have a NaN in one of these columns: ['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM'].

Save the result in `nonull_df`.

We can use `df.dropna` to do this. Pandas documentation on this method is [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html).

In [46]:
nonull_df = raw_df.dropna(subset=['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM'])

To check you've done it correctly, you could use `clean_df.isnull().sum()` to confirm that there are no NaN rows in the columns we're interested in:

In [47]:
nonull_df.isnull().sum()

PM2.5        0
year         0
month        0
day          0
hour         0
PM10        17
SO2        242
NO2        331
CO         866
O3         685
TEMP         0
PRES         0
DEWP         0
RAIN         0
wd           0
WSPM         0
station      0
dtype: int64

### Removing unwanted columns 

Let's remove the columns we're not going to be using. We can use `nonull_df.drop(list_of_column_names, axis=1)` to do this. We will drop: ['year','month','day','PM10','SO2','NO2','CO','O3','station'].

Again, feel free to check if it's worked with `clean_df.isnull().sum()`, for example.

In [48]:
clean_df = nonull_df.drop(labels= ['year','month','day','PM10','SO2','NO2','CO','O3','station'], axis=1)

In [49]:
#there should be 34284 rows left in your dataframe, and 8 columns 
clean_df.shape #=(34284, 8)

(34284, 8)

### Splitting the dataset

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. **It is important to remember that the test data has to be set aside before preprocessing**.

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage. Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Later we will be performing a grid search to select parameter values. To do this we'll do cross-validation, but rather than split the data into training and validation here we'll split it later. So for now we'll just split into:

- The training (and validation) set will have 85% of the total observations, 
- The test set, the remaining 15%.

To avoid unwanted correlations connecting the training and test, we will split these by time. So:

- Take the first 85% of the rows from clean_df and put them in train_df, take the remaining 15% of the rows and put them in test_df

In [50]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(clean_df, train_size=0.85, random_state= 10)

To check the sizes are correct, we can use:

In [51]:
len(train_df)/len(clean_df),len(test_df)/len(clean_df)

(0.8499883327499709, 0.15001166725002918)

# Detour: Lasso Regression

Later we will use the sklearn toolkit, but in this section **will develop our own code** to do the Lasso regression.

### Ordinary Least Squares Regression

First, let's just perform normal linear regression.

We'll use a toy design matrix & labels to use to check our code works. We'll also specify a weight vector too, for testing.

In [52]:
X = np.array([[0.0,0],[1,3],[2.2,3]])
y = np.array([0.0,1,2])
w = np.array([1.0,2])

###  Prediction Function

The first task is to write a function to make predictions. Complete this function for linear regression, i.e. the predictions for all our points $f(X,\mathbf{w}) = X \mathbf{w}$.

In [53]:
def prediction(X,w):
    pred = np.dot(X, w)
    return pred

In [54]:
#use this code to check you've written the right function
np.all(prediction(X,w)==np.array([0. , 7. , 8.2])) #Should return 'True'

True

### Objective Function 

Now we need to write a function that returns the 'error'.
We'll just do normal Ordinary Least Squares with linear regression, so if you remember the cost function for that is:

$$E = \sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2$$

Where $E$ is the error, $N$ the number of points, $y_i$ is one of the labels, $\mathbf{x}_i$ is the input for that label, $\mathbf{w}$ is the weight vector. $f$ is the prediction function you've already written. Or feel free to substitute in $\mathbf{x}_i^\top \mathbf{w}$.

In [55]:
def objective(X,y,w):
    """
    Computes the sum squared error (for us to perform OLS linear regression).
    """
    #Put answer here
    temp1 = prediction(X,w)
    temp2 = (y-temp1)
    Error = np.dot(temp2.T, temp2)
    return Error

In [56]:
#You can use this code to check you've written the right function
objective(X,y,w)==74.44

True

### Objective Function Gradient

Now you need to find the derivative of the objective wrt the parameter vector. You've already done this in lectures, so remember the gradient of the error function (for linear regression) is:

$$\frac{\partial E}{\partial \mathbf{w}} = 2 X^\top X \mathbf{w} - 2 X^\top y$$

Add code to do this here:

In [57]:
def objective_derivative(X,y,w):
    """
    Computes the derivative of the sum squared error, wrt the parameters.
    """
    #Put answer here   
    derivative = -2*X.T@y + 2*X.T@X@w
    return derivative

objective_derivative(X,y,w)

array([39.28, 73.2 ])

To check your gradients are correct, we can estimate the gradient numerically:

In [58]:
def numerical_objective_derivative(X,y,w):
    """
    Computes a numerical approximation to the derivative of the sum squared error, wrt the parameters.
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective(X,y,w+d)-objective(X,y,w-d))/2e-6
    return g

The two gradient vectors should be approximately equal:

In [59]:
objective_derivative(X,y,w),numerical_objective_derivative(X,y,w)

(array([39.28, 73.2 ]), array([39.28, 73.2 ]))

### Optimise $\mathbf{w}$ to minimise the objective 

Now you need to use the gradient function you've written to maximise $\mathbf{w}$ using gradient descent.
Start with a sensible choice of w. You'll need to loop lots of times (e.g. 1000). At each iteration: compute the gradient and subtract the scaled gradient from the w parameter (you'll need to scale it by the learning rate, of e.g. 0.01).

In [60]:
def optimise_parameters(X,y,startw):
    """
    Returns the w that minimises the objective.
    """
    #Put answer here
    for a in range (2000):
        grad = objective_derivative(X,y,startw)
        startw = startw - grad*0.01
    return startw

optimise_parameters(X,y,w)

array([0.83333333, 0.05555556])

In [61]:
bestw = optimise_parameters(X,y,w)
print(bestw) #print our solution

[0.83333333 0.05555556]


Let's compare this to the answer provided by sklearn:

In [62]:
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=False)
clf.fit(X,y)
print(clf.coef_) #matches the value of w we found above, hopefully!

[0.83333333 0.05555556]


## Lasso Regression

###  New Objective Function

We're now going to regularise the regression using $L_1$ regularisation - i.e. Lasso regression.

We need a **new objective function**:

$$E = \frac{1}{2N}\sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2 + \alpha \sum_{j=1}^P |w_j|$$

This is similar to before (but the first term is now half the mean squared error, rather than the sum squared error). The second term is the L1 regularisation term.

In [63]:
def objective_lasso(X,y,w,alpha):
    """
    Computes half the mean squared error, with an additional L1 regularising term. alpha controls the level of regularisation.
    """
    
    
    temp1=prediction(X,w)
    temp2 = (y-temp1)
    
    Error = (np.dot(temp2.T, temp2))/(2*len(X)) + alpha * (sum(abs(w)))
    
    return Error
    

### The gradient of the lasso regression objective

The tricky bit the derivative of the objective.

The first part is similar to before. So, with the regularising term, the derivative is:
$$\frac{\partial E}{\partial \mathbf{w}} = \frac{1}{N}(X^\top X \mathbf{w} - X^\top y) + \alpha\;\text{sign}(\mathbf{w})$$

where $\text{sign}(\mathbf{w})$ returns a vector of the same shape as $\mathbf{w}$ with +1 if the element is positive and -1 if it's negative. The `np.sign` method does this for you.

Have a think about why this is (think about what differentiating the 'absolute' function $|w_j|$ involves - think about what happens when it's positive vs when it's negative. 

In [64]:
def objective_lasso_derivative(X,y,w,alpha):
    """
    Returns the derivative of the Lasso objective function.
    """
    #Put answer here
    temp = np.sign(w)
    grad = (X.T@X@w - X.T@y)/len(X) + (alpha*temp)
    
    return grad

objective_lasso_derivative(X,y,w,0.1)

array([ 6.64666667, 12.3       ])

We can check it again, numerically. The two pairs of parameters should be the same:

In [65]:
def numerical_objective_lasso_derivative(X,y,w,alpha):
    """
    This finds a numerical approximation to the true gradient
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective_lasso(X,y,w+d,alpha)-objective_lasso(X,y,w-d,alpha))/2e-6
    return g

objective_lasso_derivative(X,y,w,0.1),numerical_objective_lasso_derivative(X,y,w,0.1)

(array([ 6.64666667, 12.3       ]), array([ 6.64666667, 12.3       ]))

### Optimise $\mathbf{w}$ to minimise the Lasso objective

As before we need to optimise to find the optimum value of $\mathbf{w}$, for this Lasso objective.
You'll need to loop lots of times (e.g. 5000). Start with a sensible choice of w. At each iteration: compute the gradient and subtract the scaled gradient from the w parameter (you'll need to scale it by the learning rate, of e.g. 0.05).

In [66]:
def optimise_parameters_lasso(X,y,startw):
    """
    Returns the w that minimises the Lasso objective.
    """
    #Put answer here
    for i in range (5000):
        grad =objective_lasso_derivative(X,y,startw,0.1)
        startw = startw - grad * 0.05
    return startw

optimise_parameters_lasso(X,y,w)


array([0.63888889, 0.14259259])

We can check against the sklearn method:

In [67]:
clf = linear_model.Lasso(alpha=0.1,fit_intercept=False)
clf.fit(X,y)
print(clf.coef_)

[0.63931263 0.1423666 ]


The above result should approximately match the one you computed.

# Back to air pollution

### One-hot-encoding

One of the columns isn't numerical, but instead is a string type: The wind direction. The best way to deal with this is one-hot-encoding.

pandas has a tool for doing this: `pd.get_dummies(series, prefix='prefix_to_use')`. In our example the series is: `clean_df.wd`.

You'll need to:
1. Make the one-hot encoding table using the code above.
2. Delete the `wd` column from our table (hint: you did this earlier for other columns).
3. Join the one hot data to the table. To do this use something like `dataframe1.join(dataframe2)`.



In [68]:
def add_wd_onehot(df):
    """Add new one-hot encoding set of columns, removes the old column it's made from. Returns new dataframe."""
     # Get one hot encoding of columns 'vehicleType'
    new_df=pd.get_dummies(df.wd, prefix= 'wd')
    # Drop column as it is now encoded
    new_clean = df.drop(labels= ['wd'], axis=1)
    # Join the encoded df
    Final_df= new_clean.join(new_df)  
    
          
    return Final_df

#you could use this code to see if it's worked?
train_df_wdencoded = add_wd_onehot(train_df)
train_df_wdencoded

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,WSPM,wd_E,wd_ENE,wd_ESE,...,wd_NNW,wd_NW,wd_S,wd_SE,wd_SSE,wd_SSW,wd_SW,wd_W,wd_WNW,wd_WSW
No,,,,,,,,,,,,,,,,,,,,,
28434,50.0,17,29.400,998.8,7.9,0.0,4.4,False,False,False,...,False,False,False,False,False,True,False,False,False,False
34258,272.0,9,-2.575,1016.5,-6.9,0.0,1.6,False,False,False,...,False,False,False,False,False,False,False,False,False,False
28462,49.0,21,26.100,1000.8,8.6,0.0,3.0,False,False,False,...,False,False,False,False,False,True,False,False,False,False
8235,75.0,2,-1.300,1022.8,-5.2,0.0,1.4,False,False,False,...,False,False,True,False,False,False,False,False,False,False
8498,44.0,1,-1.900,1028.4,-9.5,0.0,0.8,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10310,30.0,13,17.500,1010.3,-14.5,0.0,2.2,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9480,54.0,23,19.200,1009.3,-1.3,0.0,0.6,False,False,False,...,False,False,False,False,False,False,False,False,True,False
28571,83.0,10,25.700,1002.9,9.9,0.0,1.4,False,True,False,...,False,False,False,False,False,False,False,False,False,False


### Standardise the data 

Now we need to standardise the data.

You could manipulate just some columns by using, for example: `df.iloc[:,1:]` - this returns a dataframe that consists of all but the first column.

Feel free to use either tools from `sklearn.preprocessing` or standardise it by, for example, using the mean and the standard deviation of the columns by calling `some_dataframe.mean()` or `some_dataframe.std()`.

In [69]:
from sklearn.preprocessing import StandardScaler

def standardise(df):
    """
    Returns a new dataframe in which all but the PM2.5 columns are standardised (i.e. have a mean of zero and standard deviation of 1)

    Think about if you want to standardise using the *training* data's mean and standard deviation (for the test data).
    """
   
    new_df = df.iloc[:,1:]
    temp_df = df.iloc[:,0].to_frame()
    
    Final_df = (new_df - new_df.mean())/ new_df.std()  
    Final_df= temp_df.join(Final_df)  
    
    return Final_df

train_df_preprocessed = standardise(add_wd_onehot(train_df))
test_df_preprocessed = standardise(add_wd_onehot(test_df))


Here we put the training and test inputs (X) and outputs (y) into four variables:

In [70]:
X = train_df_preprocessed.iloc[:,1:].to_numpy()
y = train_df_preprocessed.iloc[:,0].to_numpy()

Xtest = test_df_preprocessed.iloc[:,1:].to_numpy()
ytest = test_df_preprocessed.iloc[:,0].to_numpy()

clf = linear_model.Lasso(alpha=0.1,fit_intercept=True)
clf.fit(X,y)

Train = clf.predict(X)
Test = clf.predict(Xtest)

We can use the same code we wrote before, using the Lasso from sklearn to fit the data. Here we'll turn fit_intercept on, as we've not added a '1's column to our design matrix.

So feel free to use:
```
clf = linear_model.Lasso(alpha=0.1,fit_intercept=True)
clf.fit(X,y)
```

### Finding the RMSE of the Lasso regressor predictions

Next compute the **RMSE** of the predictions for (a) the training data and (b) the test data.
The RMSE (root mean squared error) could be computed, for example with:

```np.sqrt(np.mean((predicted_values-true_values)**2))```

In [71]:

rmse_lasso_train = np.sqrt(np.mean((Train - y )**2))
rmse_lasso_test = np.sqrt(np.mean((Test - ytest)**2))

rmse_lasso_train, rmse_lasso_test

(74.78698078293763, 74.42378530499064)

We can compare this to the standard deviation of the data, we should do better than that!

In [72]:
np.std(y), np.std(ytest)

(85.99145204106279, 85.98333117675988)

### Random Forest 

The final step is to use a random forest regressor.

If we use the default random forest regressor, we find we get considerable over-fitting. So we need to explore different parameters. We will use a cross-validated grid search over the parameters:

- max_features: The number of features to consider when looking for the best split (i.e. controls subsampling), *from 1 to the number of features* in 4 steps (e.g. use `np.linspace`)
- n_estimators: The number of trees in the forest, from 10 to 100 in 4 steps.
- max_samples : the number of samples to draw from to train each base estimator, from 0.1 to 0.9 in 4 steps.

We will use `GridSearchCV`.

Have a look at the documentation for this, the three parameters we need to specify are:

- the 'estimator': an **INSTANCE** of RandomForestRegressor.
- param_grid: a **DICTIONARY**, each item is the title of the parameter, and equals an array of the values we need to test. For exmaple, one of the items might be `{'max_samples': np.linspace(0.1, 0.9, 5)}`.
- You'll need to think carefully how to make the lists for the `max_features` and `n_estimators` as these both need to be (positive) integers. E.g. use `.astype(int)`.

In [73]:
#1. Create a grid of parameter values for n_estimators, max_features and max_samples,

max_features=np.linspace(1, X.shape[1], 4).astype(int)
n_estimators=np.linspace(10, 100, 4).astype(int)
max_samples=np.linspace(0.1, 0.9, 4)

#2. Create a GridSearchCV object, using the random forest regressor:
param_grid = dict(n_estimators = n_estimators,  max_samples=max_samples , max_features = max_features)
grid_regression = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, scoring='neg_mean_squared_error')


#Note: Because there is so much training data, using the full dataset takes too long. So here we'll just use 10%
np.random.seed(42)
idx = np.sort(np.random.choice(len(X), size=int(len(X)*0.1), replace=False))
#3. Fit to training data in (the subset of) X and y
grid_regression.fit(X[idx,:],y[idx])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 1,  8, 15, 22]),
                         'max_samples': array([0.1       , 0.36666667, 0.63333333, 0.9       ]),
                         'n_estimators': array([ 10,  40,  70, 100])},
             scoring='neg_mean_squared_error')

Here we print the best parameters from the grid search (on the training/validation cross-validation run):

In [74]:
best_n_estimators = grid_regression.best_params_['n_estimators']
best_max_features = grid_regression.best_params_['max_features']
best_max_samples = grid_regression.best_params_['max_samples']
print('Best n_estimator', best_n_estimators)
print('Best max features', best_max_features)
print('Best max samples', best_max_samples)

Best n_estimator 100
Best max features 15
Best max samples 0.6333333333333333


### RMSE for the Random Forest Regressor 

Finally compute the RMSE for the training and test data:

In [75]:
Training = RandomForestRegressor(n_estimators=best_n_estimators,max_samples=best_max_samples,max_features = best_max_features)
Training.fit(X,y)

T = Training.predict(X)
y_pred = Training.predict(Xtest)


rmse_rf_train = np.sqrt(np.mean((T - y )**2))
rmse_rf_test = np.sqrt(np.mean((y_pred- ytest)**2))

print(rmse_rf_train), print(rmse_rf_test)

31.36099223867163
59.764234244276764


(None, None)

We can compare this to the standard deviations for the two sets of data.

In [76]:
np.std(y), np.std(ytest)

(85.99145204106279, 85.98333117675988)

### Did the random forest do better than lasso regression?



In [79]:
#Yes, The RMSE for random forest is much less. However both model did well when RMSE is compared to standard deviation